In [1]:
import numpy as np

def generate_values(mean, std_dev, num_values=5, decimals=3):
    # Generate the values using normal distribution with the specified mean and standard deviation
    values = np.random.normal(loc=mean, scale=std_dev, size=num_values)

    # Round values to the specified number of decimals
    rounded_values = [round(val, decimals) for val in values]

    return rounded_values

# Example usage:
mean = 54.251
std_dev = 0.00876163964572566
generated_values = generate_values(mean, std_dev)
print("Generated Values:", generated_values)
def calculate_standard_deviation(values):
    mean = sum(values) / len(values)

    # Calculate the variance
    variance = sum((x - mean) ** 2 for x in values) / len(values)

    # Calculate the standard deviation
    std_dev = variance ** 0.5

    return std_dev

# Example usage#93.357
values = [76.711, 76.702, 76.722, 76.698, 76.732]
std_dev = calculate_standard_deviation(values)
print("Standard Deviation (Manual Calculation):", std_dev)


Generated Values: [54.263, 54.264, 54.238, 54.248, 54.242]
Standard Deviation (Manual Calculation): 0.012585706178042748


In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the images
img1 = cv2.imread('/content/cameraman.tif', cv2.IMREAD_GRAYSCALE)  # Ensure this path is correct
img2 = cv2.imread('/content/text.tif', cv2.IMREAD_GRAYSCALE)  # Ensure this path is correct

# Check if images are loaded
if img1 is None or img2 is None:
    raise Exception("Could not load one or both images. Check the file paths.")

# Resize img2 to match img1's dimensions if necessary
if img1.shape != img2.shape:
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_AREA)

# Blend the images
# Adjust alpha (img1 weight) and beta (img2 weight) to change visibility preference
alpha = 0.6
beta = 0.4
blended_image = cv2.addWeighted(img1, alpha, img2, beta, 0)

# Display the result
cv2_imshow( blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally, save the result to a file
cv2.imwrite('content/blended_image.tif', blended_image)



import cv2
import numpy as np

# Load the images
img1 = cv2.imread('/content/cameraman.tif', cv2.IMREAD_GRAYSCALE)  # Ensure the path is correct
img2 = cv2.imread('/content/text.tif', cv2.IMREAD_GRAYSCALE)       # Ensure the path is correct

# Check if images are loaded
if img1 is None or img2 is None:
    raise Exception("Could not load one or both images. Check the file paths.")

# Resize img2 to match img1's dimensions if necessary
if img1.shape != img2.shape:
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_AREA)

# Convert images to double (float64 in numpy)
img1_double = img1.astype(np.float64)
img2_double = img2.astype(np.float64)

# Scale img2 by 255 and add to img1
result = img1_double + 255 * img2_double

# Convert the result back to uint8
result_uint8 = np.clip(result, 0, 255).astype(np.uint8)

# Display the result
cv2_imshow( result_uint8)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally, save the result to a file
cv2.imwrite('content/superimposed_image.tif', result_uint8)


In [ ]:
import cv2
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt

# Load the image
img = cv2.imread('/content/cameraman.tif', cv2.IMREAD_GRAYSCALE)

# a. Roberts
kernel_roberts_x = np.array([[1, 0], [0, -1]])
kernel_roberts_y = np.array([[0, 1], [-1, 0]])
roberts_x = cv2.filter2D(img, -1, kernel_roberts_x)
roberts_y = cv2.filter2D(img, -1, kernel_roberts_y)
roberts = roberts_x + roberts_y

# b. Prewitt
kernel_prewitt_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
kernel_prewitt_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
prewitt_x = cv2.filter2D(img, -1, kernel_prewitt_x)
prewitt_y = cv2.filter2D(img, -1, kernel_prewitt_y)
prewitt = prewitt_x + prewitt_y

# c. Sobel
sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
sobel = cv2.magnitude(sobel_x, sobel_y)

# d. Laplacian
laplacian = cv2.Laplacian(img, cv2.CV_64F)

# e. Zero-crossings of a Laplacian
laplacian = cv2.Laplacian(img, cv2.CV_64F)
sign_changes = np.logical_xor(laplacian[:-1] < 0, laplacian[1:] > 0)
zero_crossings = np.zeros_like(laplacian)
zero_crossings[:-1][sign_changes] = 255  # Mark zero crossings with white

# f. Marr-Hildreth using Gaussian Laplacian
sigma = 2
gaussian_laplace = ndimage.gaussian_laplace(img, sigma=sigma)
marr_hildreth = np.zeros_like(gaussian_laplace)
marr_hildreth[gaussian_laplace < 0] = 255

# g. Canny
canny = cv2.Canny(img, 100, 200)

# Plot all the results
titles = ['Original Image', 'Roberts', 'Prewitt', 'Sobel', 'Laplacian', 'Zero-crossings', 'Marr-Hildreth', 'Canny']
images = [img, roberts, prewitt, sobel, laplacian, zero_crossings, marr_hildreth, canny]

plt.figure(figsize=(12, 8))
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.xticks([]), plt.yticks([])

plt.show()
